In [1]:
import pandas as pd
import numpy as np
import datetime
import chardet
import time
import os

In [2]:
def neue_Spalte(DF, Spalte1, Spalte2, neue_Spalte1, neue_Spalte2):
    """Diese Funktion soll durch Kombination von zwei Spalten eine neue erstellen.
    Dann wird diese Spalte vom Ende des DataFrames am Anfang eingefügt."""

    # Erstellung der neuen Spalte als Kombination von Spalte1 und Spalte2
    DF[neue_Spalte1] = DF[[Spalte1,Spalte2]].astype(str).agg("/".join, axis = 1)
    # Die neue Spalte wird vom Ende des DataFrames am Anfang eingefügt. 
    Drop_Spalte = DF.pop(neue_Spalte1)
    DF.insert(2, neue_Spalte2, Drop_Spalte)
    # Das neue Dataframe wird ausgefürt 

    return DF

In [3]:
def Funktion_B14_02(DF, Spalte1, Spalte2):
    """Diese Funktion sucht in DF-230/DSDW0230 nach den Nace-Codes "K64".
      Wenn gefunden, wird in der Spalte B14 an Pos.2 "N" durch "Y" an der zweiten Stelle ersetzt."""

    DF.loc[DF[Spalte1].str.contains("K64.", na = False), Spalte2] = DF[Spalte2].str[:1] + "Y" + DF[Spalte2].str[2:]

    return DF


In [4]:
def Funktion_B14_Pos03(DF1, DF2, Spalte1, Spalte2, Spalte3, Spalte4):
    """Diese Funktion prüft ob, die Werte "CIN", "INS", "INSO", "LI", "NLI", "RI" in der Spalte DF-3386/TPDW3386 vorhanden sind.
       Falls ja, dann wird in der Spalte B14 an Pos.3,  "N" durch "Y" an der zweiten Stelle ersetzt."""

    DF_merged = DF1.merge(DF2, left_on = Spalte1, right_on = Spalte2, how = "left")
    DF_merged.loc[DF_merged[Spalte3].isin(["CIN", "INS", "INSO", "LI",
                                           "NLI", "RI"]), Spalte4] = DF_merged[Spalte4].str[:2] + "Y" + DF_merged[Spalte4].str[3:]

    return DF_merged[DF1.columns]

In [5]:
def Funktion_B14_05(DF1, DF2, Spalte1, Spalte2, Spalte3, Spalte4):
    """Diese Funktion prüft ob, die Werte "CGOV", "LGOV", "CGOV" in der Spalte DF-3386/TPDW3386 vorhanden sind.
       Falls ja, dann wird in der Spalte B14 an Pos.5,  "N" durch "Y" an der fünften Stelle ersetzt."""

    # Merge DF1 und DF2 um die Werte in Spalte DF-3386/TPDW3386 zu prüfen und "N" durch "Y" zu ersetzen
    DF_merged = DF1.merge(DF2, left_on = Spalte1, right_on = Spalte2, how = "left")
    DF_merged.loc[DF_merged[Spalte3].isin(["CGOV", "LGOV", "SGOV"]), Spalte4] = DF_merged[Spalte4].str[:4] + "Y" + DF_merged[Spalte4].str[5:]

    return DF_merged[DF1.columns]


In [6]:
def Funktion_B14_06(DF, Spalte1, Spalte2):
    """Diese Funktion sucht in DF-230/DSDW0230 nach den Nace-Codes, die "K66" enthalten.
       Falls gefunden, wird in der Spalte B14 an Pos.6 "N" durch "Y" an der sechsten Stelle ersetzt."""

    DF.loc[DF[Spalte1].str.contains("K66.", na = False), Spalte2] = DF[Spalte2].str[:5] + "Y" + DF[Spalte2].str[6:]

    return DF

In [7]:
def Funktion_B14_07(DF1, DF2, Spalte1, Spalte2, Spalte3, Spalte4):
    """Diese Funktion sucht in TPDW3386  nach dem Wert "PEN".
       Falls gefunden, wird in der Spalte B14 an Pos.7 "N" durch "Y" an der siebten Stelle ersetzt."""
    # Bedingung um zu prüfen, ob "PEN" vorhanden ist und "N" durch "Y" ersetzen
    DF_merged = DF1.merge(DF2, left_on = Spalte1, right_on = Spalte2, how = "left")
    DF_merged.loc[DF_merged[Spalte3] == "PEN", Spalte4] = DF_merged[Spalte4].str[:6] + "Y" + DF_merged[Spalte4].str[7:]

    return DF_merged[DF1.columns]

In [8]:
def Funktion_B14_Pos_10(DF1, DF2, Spalte1, Spalte2, Spalte3):
    """Diese Funktion prüft, ob ein Kunde in der Money Laundery Entittät vorhanden ist,
    und tauscht in der Unterposition Pos.10 von B14 "N" mit "Y", 
    falls die Prüfung positiv ist"""

    # Merge der beiden Tabelle basierend auf Übereinstimmung in der Spalte1
    DF_merged = DF1.merge(DF2, left_on = Spalte1, right_on = Spalte2, how = "inner")
    #Ersetze "N" durch "Y" in Spalte2 entsprechend des Merge
    DF1.loc[DF_merged.index, Spalte3] = DF1.loc[DF_merged.index, Spalte3].str[:9] + "Y" + DF1.loc[DF_merged.index, Spalte3].str[10:]

    return DF1

In [9]:
def Funktion_B16(DF1, DF2, Spalte1, Spalte2, Spalte3, Spalte4):
    """Diese Funktion prüft, ob "I-NP" in der Spalte "TPDW0235" vorhanden ist. 
    Wenn ja, schreibt die Funktion entsprechend in B16 10 an"""

    DF_merged = DF1.merge(DF2, left_on=Spalte1, right_on=Spalte2, how="inner")

    # Wandele die Werte in Spalte3 von DF_merged in Strings um, falls sie nicht bereits vom Typ String sind
    DF_merged[Spalte3] = DF_merged[Spalte3].astype(str)

    # Verwende jetzt eine List Comprehension, um über die String-Werte zu iterieren
    DF_merged = DF_merged.reset_index(drop=True)  # Setze den Index neu, um mögliche Probleme mit den Indizes zu vermeiden
    DF1[Spalte4] = DF_merged[Spalte3].apply(lambda x: '10' if x == 'I-NP' else 'N')

    return DF1[Spalte4]

In [10]:
def B_Satz2_Zeile_aufsuchen(Tab,spalte1,value):
    """Diese Funktion wählt durch Filtrierung eine Zeile aus B_Satz2 aus
    und setzt diese Zeile unter der Zeile aus dem A_Satz. Damit man die Datensätze entsprechend der Tabellendarstellung 
    der Einreichertdatei. Value ist das Suchkriterium im B_Satz2 wie z.B die Kundennummer."""

    # Zeile aus A_Satz
    erste_Zeile = Tab[Tab[1]=="A"]
    # gewünschte Zeile aus B_Satz2
    zweite_Zeile = Tab[(Tab[1]=="B") & (Tab[spalte1]==value)]
    # Stapeln der Zeilen
    DF = pd.concat([erste_Zeile, zweite_Zeile], axis = 0)

    return DF

In [11]:

def naechsterwert2(DF, Spalte1, Spalte2):
    """Diese einfache Funktion gibt dir den nächsten in einer Tabelle"""

    return DF[Spalte2][DF[Spalte1].isin(DF[Spalte1])]

In [12]:
def func_neu_C4(DF1, DF2,Spalte1, Spalte2, Spalte3):
    """ Diese Funktion........."""

    DF_merged = DF1.merge(DF2, left_on = Spalte1, right_on = Spalte2, how = "left")
    DF_merged["A"] = "N"
    DF_merged.loc[(DF_merged[Spalte3] == "BO"), "A"] = "Y"

    Ergebnisspalte = DF_merged["A"]

    return Ergebnisspalte

In [13]:
def func_neu_C5(DF1, DF2, Spalte1, Spalte2, Spalte3):
    """Diese Funktion......"""

    DF_merged = DF1.merge(DF2, left_on = Spalte1, right_on = Spalte2, how = "left")
    DF_merged["B"] = 0
    DF_merged.loc[(DF_merged[Spalte3] == "CR"), "B"] = 1

    # Group data
    DF_grouped = DF_merged.groupby(Spalte1)["B"].sum().reset_index()

    return DF_grouped["B"]

In [14]:
def Funktion_C15(DF1, DF2, Spalte1, Spalte2, Spalte3, Spalte4):
    """Diese Funktion ....."""

    merged = DF1.merge(DF2, left_on=Spalte1, right_on=Spalte2, how="left")

    condition = merged[Spalte3] == "0"

    DF1[Spalte4] = "N"
    DF1.loc[condition, Spalte4] = "Y"

    return DF1[Spalte4]

In [15]:
def Funktion_C27(row):
    """Diese Funktion ist definiert entsprechend der Spezifikation"""

    # Wert: 90 - Ohne Ausschluss
    if all(row['B14'] == 'N') and all(row['C20'] == 'N'):
        return '90'

    # Wert: 01 - Ausschluss nur nach EinSiG
    if any(row['B14'][0:15] == 'Y') and all(row['B14'][15:50] == 'N') and all(row['C20'][10:50] == 'N'):
        return '01'
    elif any(row['C20'][0:10] == 'Y') and all(row['C20'][10:50] == 'N') and all(row['B14'][15:50] == 'N'):
        return '01'

    # Wert: 20 - Ausschluss nur nach ESF-Statut
    if any(row['B14'][30:50] == 'Y') and all(row['B14'][0:30] == 'N') and all(row['C20'][0:30] == 'N'):
        return '20'
    elif any(row['C20'][30:50] == 'Y') and all(row['C20'][0:30] == 'N') and all(row['B14'][0:30] == 'N'):
        return '20'

    # Wert: 11 - Ausschluss "Bagatellgrenze" nach EinSiG und ESF-Statut
    if all(row['C20'][1:49] == 'N') and row['C20'][49] == 'Y':
        return '11'

    # Wert: 10 - Ausschluss nach EinSiG und ESF-Statut (alle anderen Fälle)
    return '10'

In [ ]:
path = r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\DE1D_2023_06_13"

In [1]:
DF_BP = pd.read_csv(r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\DE1D_2023_06_13\BPART.csv", sep = "\t", skiprows = 9)
# KYDW0007 wird durch eine "Join" der Spalten KDW0007 und KDW0007O. Die Spalte wird am Ende des Dataframes hinzugefügt 
DF_BP["KYDW0007"] = DF_BP[["KDW0007O","KDW0007"]].astype(str).agg("/".join, axis = 1)
# Die neue KYDW0007-Spalte wird am Anfang des Dataframes hingefügt 
first_column = DF_BP.pop("KYDW0007")
DF_BP.insert(2, "KYDW0007O", first_column)
# Das neue Dataframe wird ausgefürt 

NameError: name 'pd' is not defined

In [ ]:
# Path zur Directory, wo alle einzulesenden Dateien vorhanden sind

path = r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\DE1D_2023_06_13"

##########################################-I Einlesen Der Daten-####################################################################################
# Einlesen der Business Partner und Business Partner act in Liability und die Money Laundery-Entitäten. 
# Dateien, die wir für den Aufbau des B-Satzes benötigen
# Nach dem Einlesen muss entsprechend der Entitäten die neue Spalte KYDW0007, KYDWR50880 und KYDWR5469O erzeugt werden,  entsprechend den Testdaten
# Z.B KYDW0007 ist eine Kombination der Spalten KDW0007 und KDW0007O

# 1. Einlesen der Business Partner-Entität

DF_BP = pd.read_csv(r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\DE1D_2023_06_13\BPART.csv", sep = "\t", skiprows = 9)
# KYDW0007 wird durch eine "Join" der Spalten KDW0007 und KDW0007O. Die Spalte wird am Ende des Dataframes hinzugefügt 
DF_BP["KYDW0007"] = DF_BP[["KDW0007O","KDW0007"]].astype(str).agg("/".join, axis = 1)
# Die neue KYDW0007-Spalte wird am Anfang des Dataframes hingefügt 
first_column = DF_BP.pop("KYDW0007")
DF_BP.insert(2, "KYDW0007O", first_column)
# Das neue Dataframe wird ausgefürt 

print(DF_BP)

# Einlesen der Datei  "Datei_TPDW3386", die die Spalte TPDW3386 enthält.Diese Spalte wird benötigt, damit man die Codes der Unterpositionen  Pos.05 und Pos.7 der B14-Spalte ausführen kann. 
# In "Datei_TPDW3386" heisste die Spalte "TPGC33886". Die Spalte wird jedoch in den DataFrame "DF_BP" als "TPDW3386" eingefügt.

# Einlesen der Datei mit der Spalte TPDW3386.

with open(r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\DE1D_2023_06_13\Datei_TPDW3386.csv", "rb") as f:
    enc = chardet.detect(f.read())

DF_3386 = pd.read_csv(r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\DE1D_2023_06_13\Datei_TPDW3386.csv", sep =";", encoding = enc["encoding"])

print(DF_3386)

# Werte der Spalte TPGC3386
DF_3386["TPGC3386"].values

# Einfügen der Spalte "TPGC3386" als "TPDW3386" in DF_BP

DF_BP["TPDW3386"] = DF_3386["TPGC3386"]

print(DF_BP)

# 2.Einlesen der Business Partner act in Liability-Entität 

DF_BPact = pd.read_csv(r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\DE1D_2023_06_13\BPART_Act.csv", sep = "\t", skiprows = 9)
# KYDWR5088O wird durch eine "Join" der Spalten KDWR5088 und KDWR5088O. Die Spalte wird am Ende des Dataframes hinzugefügt 
DF_BPact["KYDWR5088"] = DF_BPact[["KDWR5088O","KDWR5088"]].astype(str).agg("/".join, axis = 1)
# Die neue KYDWR5088O-Spalte wird am Anfang des Dataframes hingefügt 
Second_column = DF_BPact.pop("KYDWR5088")
DF_BPact.insert(2,"KYDWR5088O", Second_column)
# Das neue Dataframe wird ausgefürt 

print(DF_BPact)

# Im B-Satz sollten die Kunden als Creditor vorhenden sein. Aus diesem Grund verbinden wir die Business Partner Entität und die Business Partner act in Liability. 
# Dann filtrieren wir die den gemerged Dataframe auf die Kundennummer versehen mit "CR". Die erhaltenen Kundennummern mit "CR" werden dann 
# Die erhaltenen Kundennummern mit "CR" werden dann in den B_Satz in die Spalte "B2" eingefügt.

# Merge Business Partner und Business Partner act in Liability Entität

DF_merged2 = DF_BPact.join(DF_BP, lsuffix = "KDWR5088O", rsuffix = "KYDW0007")

print(DF_merged2)

# Erstelle die Spalte der Kundennummern mit "CR"
DF_merged2 = DF_BPact.join(DF_BP, lsuffix = "KDWR5088O", rsuffix = "KYDW0007")
DF_merged2.loc[DF_merged2["TPDW5090"] == "CR"]["KYDW0007O"]

#3.Einlesen der Money laundering Entität 

DF_MONLA = pd.read_csv(r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\DE1D_2023_06_13\MONEYLA.csv", sep = "\t", skiprows = 9)
DF_MONLA["KYDWR5469"] = DF_MONLA[["KDWR5469O","KDWR5469"]].astype(str).agg("/".join, axis = 1)
neue_Spalte = DF_MONLA.pop("KYDWR5469")
DF_MONLA.insert(2,"KYDWR5469O", neue_Spalte)

DF_MONLA

# 4 Einlesen der Liability Entität für den Bau des C-Satzes

DF_Liab = pd.read_csv(r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\DE1D_2023_06_13\PLIAB.csv", sep ="\t", skiprows = 9, decimal = ",")

DF_Liab

# 5 Einlesen der PRODUCT-Entität für die Bestimmung der Produktnamen in C6

with open(r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\DE1D_2023_06_13\PRODUCT.csv", "rb") as f:
    enc = chardet.detect(f.read())

DF_PRO = pd.read_csv(r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\DE1D_2023_06_13\PRODUCT.csv", sep =";", skiprows = 9, encoding = enc["encoding"])

DF_PRO

# 6 Einlesen der Cancellation of Liability Entität für C26

BP_CANLA = pd.read_csv(r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\DE1D_2023_06_13\BP_CANLA.csv", sep ="\t", skiprows = 9, decimal = ".")

BP_CANLA

In [3]:
A_Satz = pd.DataFrame([['']*10], columns=['A{}'.format(i) for i in range(1, 11)])
A_Satz["A1"] = 'A'
A_Satz["A2"] = '165'
A_Satz["A3"] = '10'
A_Satz["A4"] = 100000.00
A_Satz["A5"] = 0.00
A_Satz["A6A"]= 1330170000.00
A_Satz["A6B"] = 5000000
A_Satz["A6C"] = 50000000
A_Satz["A7"] = 1392530000.00
A_Satz["A8"] = '00000000'
A_Satz["A9A"] = 0.00
A_Satz["A9B"] = 0.00
A_Satz["A9C"] = 0.00
A_Satz["A10"] = '00000000'

A_Satz.columns = ["A1", "A2", "A3", "A4", "A5", "A6", "A6A",
                  "A6B", "A6C", "A7", "A8", "A9", "A9A", "A9B", "A9C", "A10"]

NameError: name 'pd' is not defined

In [ ]:
B_Satz2 = pd.DataFrame([['']*16], index = range(1,1460612), columns=["B{}".format(i) for i in range(1, 17)])
B_Satz2["B1"] = "B"
B_Satz2["B2"] = DF_merged2.loc[DF_merged2["TPDW5090"] == "CR"]["KYDW0007O"]
B_Satz2["B3"] = DF_BP["DSDW0010"]
B_Satz2["B4"] = DF_BP["TPDW4438"]
B_Satz2["B5"] = ""                                    # LEER lassen
B_Satz2["B6"]= DF_BP["TPDW4438"]
B_Satz2["B7"] = DF_BP["DSDW3702"]
B_Satz2['B8'] = ""
B_Satz2["B9"] = DF_BP["DSDW0013"]
B_Satz2["B10"] = DF_BP["DSDW0014"]
B_Satz2["B11"] = DF_BP["TPDW0012"]
B_Satz2["B12"] = DF_BP["DTDW4315"]
B_Satz2["B13"] = DF_BP["DSDW0230"]
B_Satz2["B14"] = "N"*50
B_Satz2["B15"] = "X"
B_Satz2["B16"] = Funktion_B16(B_Satz2, DF_BP, "B2", "KYDW0007O", "TPDW0235", "B16")

B_Satz2

In [ ]:
B_Satz2 = Funktion_B14_Pos03(B_Satz2, DF_BP, "B2", "KYDW0007O", "TPDW3386", "B14")

B_Satz2

# Für die Unterposition B14_Pos.04 schreiben wir den festen Wert "N"

# Aufruf der Funktion für Position B14_Pos.05."CGOV", "LGOV" und "SGOV" werden in TPDW3386 geprüft
# und entsprechend "N" durch "Y" an der 5. Stelle ersetzt

B_Satz2 = Funktion_B14_05(B_Satz2, DF_BP, "B2", "KYDW0007O", "TPDW3386", "B14")

B_Satz2

# Aufruf der Funktion für Position B14_Pos.06. String, die "K66." werden in DSDW0230 geprüft
# und entsprechend "N" durch "Y" an der 6. Stelle ersetzt

B_Satz2 = Funktion_B14_06(B_Satz2, "B13", "B14")

B_Satz2

# Aufruf der Funktion für Position B14_Pos.07. String, die "PEN" werden in "TPDW3386" geprüft
# und entsprechend "N" durch "Y" an der 7. Stelle ersetzt

B_Satz2 = Funktion_B14_07(B_Satz2, DF_BP, "B2", "KYDW0007O", "TPDW3386", "B14")

B_Satz2

# Aufruf der Funktion für Position B14_Pos.10. String, die "PEN" werden in "TPDW3386" geprüft
# und entsprechend "N" durch "Y" an der 10. Stelle ersetzt

B_Satz2 = Funktion_B14_Pos_10(B_Satz2, DF_MONLA,"B2","KYDWR5469O", "B14")

B_Satz2

# Aussehen der Spalte B14 nach den Transformationen

B_Satz2.B14

# Prüfen der Werte in B14

B_Satz2.B14.value_counts()

In [ ]:
DF1 = pd.DataFrame(B_Satz2)
DF1.to_csv("B_Satz.csv")
DF1 = pd.read_csv("B_Satz.csv", header = None)
DF1 = DF1.tail(-1)
DF1

# A_Satz wird noch als DataFrame eingelesen und alle Spaltennamen werden entfernt

DF2 = pd.DataFrame(A_Satz)
DF2.to_csv("A_Satz.csv")
DF2 = pd.read_csv("A_Satz.csv", header = None)
DF2
DF2 =DF2.tail(-1)
DF2

# Spezieller Code damit wir die Zeilen aus A_Satz und Zeilen aus dem B_Satz untereinander stapeln können.

# Wir definiere einen neuen DataFrame, der die Spalten von DF1 und DF2 hat.
# Hier bekommt DF2 einen neuen index so dass, die Spalten von DF2 in der gleichen Reihenfolge wie die Spalte von DF1 sind
# Das heisst, dass neue Spalten mit None gefüllt werden in DF2 eingefügt
DF_GroßerSatz = pd.concat([DF2, DF1.reindex(columns=DF1.columns)], ignore_index=True)
# Entferne die erste Spalte
DF_GroßerSatz = DF_GroßerSatz.iloc[:, 1:]

# DF_GroßerSatz enthält jetzt die Daten aus beiden DataFrames, wobei fehlende Spalten in DF2 auf None gesetzt sind

DF_GroßerSatz


# DF_GroßerSatz in einer CSV_Datei

DF_GroßerSatz.to_csv(r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\05_EinSiG_Coding\Python_Code\DF_GroßerSatz.csv")


# Dataframe bestehend nur aus alleinigen Zeilen des A_ und B_Satzes

DataFrame_A_B = B_Satz2_Zeile_aufsuchen(DF_GroßerSatz,2,"TRSRYDE1-BP/104140197")

DataFrame_A_B

# DataFrame_A_B in einer CSV_Datei

DataFrame_A_B.to_csv(r"\\FSDEBSV00630\sap_daten\DE-Testdaten_SSW_Meldewesen\TP6_Ad-hoc-Reporting\05_EinSiG_Coding\Python_Code\Datei_A_B.csv")

###################++++ 3. C-Satz ++++++++++#######################################

################++++++ Definition des C-Satzes(Liability) +++++##############

# Merge der Liability und Product Entitäten für das Datenfel C7

DF_merg3 = DF_Liab.merge(DF_PRO, left_on = "KDWR5097", right_on = "KMDPRODID", how = "left")

DF_merg3

# Merge der Cancellation- und Liability-Entität für C26

DF_merged10 = DF_Liab.merge(BP_CANLA, left_on = "KDW5082", right_on = "KDWR5000", how = "left")
DF_merged10


C_liab = pd.DataFrame([['']*27], index = range(0,1460610), columns=["C{}".format(i) for i in range(1, 28)])
C_liab["C1"] = "C"
C_liab["C2B"] = DF_Liab["KDW5082"]
C_liab.dropna(subset = "C2B", inplace=True)
DF = DF_BPact[DF_BPact["TPDW5090"] == "CR"]
DF = DF[["KDWR5089", "KYDWR5088O"]]
C_liab = C_liab.merge(DF, left_on = "C2B", right_on = "KDWR5089", how = "left")
C_liab["C4"] = func_neu_C4(DF_Liab, DF_BPact, "KDW5082",  "KDWR5089", "TPDW5090")
C_liab["C5"] = func_neu_C5(DF_Liab, DF_BPact, "KDW5082", "KDWR5089", "TPDW5090")
C_liab["C6"] = naechsterwert2(DF_Liab, "KDW5082", "DTDW5113").apply(lambda x: str(int(x)) if not pd.isnull(x) else x)              # ["DTDW5113" for "DTDW5113" in DF_Liab.set_index("KDW5082").columns]  # DF_Liab["DTDW5113"].sort_values("KDW5082")                                        #S-Verweis: Suche aus PLIABA den Vertrag (KYDW5082) und das dazugehörige Datum DTDW5113 und schreibe es in C6
C_liab["C7"] = naechsterwert2(DF_merg3,"KDW5082", "DSMDPRODN")                                                                   # Sverweis(DF_Liab,DF_PRO, "KDW5082", "KDWR5097", DF_Liab, DF_PRO,"KMDPRODID", "TPMDPPRCD", "KDW5082" )
C_liab["C8"] = naechsterwert2(DF_Liab, "KDW5082", "UCDW5105")
C_liab["C9"] = naechsterwert2(DF_Liab, "KDW5082", "ATDW5105")                                                         # naechsterwert2(DF_Liab, "KDW5082", "ATDW5105").apply(lambda x:-x)                                                            # DF_Liab[DF_Liab["KDW5082"].isin(DF_Liab["KDW5082"])]DF_Liab["ATDW5105"].apply(lambda x: -x if x > 0 else x)
C_liab["C10"] = "{:,.5f}".format(100000/100000)                                                                        # 1.00000                                                            #naechsterwert2(DF_Liab, "KDW5082", "TPDW5086")
C_liab["C11"] = C_liab["C9"].astype(float) * C_liab["C10"].astype(float)
C_liab["C12"] = naechsterwert2(DF_Liab, "KDW5082", "QUDW5537").astype(str) + DF_Liab["UCDW5537"]
C_liab["C13"] = "X"
C_liab["C14"] = naechsterwert2(DF_Liab, "KDW5082", "DTDW5084")
C_liab["C15"] = Funktion_C15(C_liab, DF_Liab, "C2B", "KDW5082", "QUDW5182", "C15")
C_liab["C16"] = naechsterwert2(DF_Liab, "KDW5082", "TPDW5562")
C_liab["C17"] = naechsterwert2(DF_Liab, "KDW5082", "ATDW5107")
C_liab["C18"] =  C_liab["C10"].astype(float) * C_liab["C17"].astype(float)
C_liab["C19"] =  C_liab["C18"].astype(float) + C_liab["C11"].astype(float)                               # C_liab["C11"] + C_liab["C18"]
C_liab["C20"] = "N"*50                                                                     # C_Pos[["C_Pos{}".format(i) for i in range(1, 51)]].astype(str).agg("".join, axis = 1)
C_liab["C21"] = "N"*50                                                        # C_Pos[["C_Pos{}".format(i) for i in range(1, 51)]].astype(str).agg("".join, axis = 1)
C_liab["C22"]= "DE"
C_liab["C23"] = ""
C_liab["C24"] = naechsterwert2(DF_Liab, "KDW5082", "DTDW5484")
C_liab["C25"] = naechsterwert2(DF_Liab, "KDW5082", "QUDW5094")
C_liab["C26"] = naechsterwert2(DF_merged10, "KDW5082", "ATDW5156")
C_liab["C27"] = "X"                   # B_C_merged.apply(Funktion_C27, axis = 1)

C_liab.rename(columns = {"KYDWR5088O": "C2A"}, inplace = True)

C_liab

# Auswählen der Spalten, die in C-Satz vorhanden sein müssen und in der
# richtigen Reihenfolge

C_Satz = C_liab[["C1", "C2A", "C2B", "C3", "C4", "C5", "C6", "C7", "C8", "C9", "C10", "C11", "C12", "C13",
                 "C14", "C15", "C16", "C17", "C18", "C19", "C20", "C21", "C22", "C23", "C24", "C25", "C26", "C27"]]
# gewünschte C-Satz ausgeben

C_Satz

#####++++Code für Gemeinschaftskonten++++##########################

Tab2 = C_Satz.groupby("C2B")

#
Tab3 = Tab2["C2A"].count().reset_index(name = "Anzahl Vertrag pro Kunde").sort_values(["Anzahl Vertrag pro Kunde"],ascending = False)
Tab3

#
Tab4 = Tab3.loc[Tab3["Anzahl Vertrag pro Kunde"] == 2]
Tab4

#
Tab5 = C_liab.loc[C_liab["C2B"].isin(Tab4["C2B"])]
Tab5

#
Tab6 = Tab5.groupby("C2B")["C2A"].apply(list).reset_index(name = "Kunden_Nummern")
Tab6

#
Tab7 = Tab6["C2B"].tolist()
Tab7

#
Num = Tab7[0]
Num

#
Tab8 = C_liab.loc[C_liab["C2B"] == Num]
Tab8

#
Tab9 = Tab8.groupby("C2A")["C2B"].apply(list).reset_index(name = "Vertrag_Nummern")
Tab9

#

C_Satz.loc[(C_Satz["C2A"] == '155093225') | (C_Satz["C2A"] == '21892380')]

################++++++ Definition des C-Satzes(Contract) +++++##############

In [ ]:
C_Satz.to_csv("C.csv", index=False)
B_Satz2.to_csv("C.csv", index=False)
A_Satz.to_csv("C.csv", index=False)